# Flip-contrained SCA

For SCA dynamics, we plot a sample of the modified Hamiltonian in "Step-Energy graph".  For flip-constrained SCA dynamics, we plot a sample of the (original) Hamiltonian in "Step-Energy graph".

## Annealing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
#import sys
#sys.path.append('../python')
#import simulator
import simulatorWithCpp as simulator
import math

%matplotlib inline
np.set_printoptions(threshold=16, edgeitems=8)

In [ ]:
MaxSteps = int(3.e3)
MaxTrials = int(1.e1)
NumNodes = 256
SeedForConfiguration = 1024

In [ ]:
isingModel = simulator.IsingModel({node: 0.e0 for node in range(NumNodes)}, {})
isingModel.SetSeed(SeedForConfiguration)
isingModel.GiveSpins(simulator.ConfigurationsType.Uniform)
InitialConfiguration = isingModel.Spins

def TryExperimentFor(isingModel, initialTemperature):
    minimumEnergiesData = np.empty(0, dtype=np.float)
    samples = np.empty((MaxSteps + 1, 4), dtype=np.float)
    #isingModel.Write()
    for i in range(MaxTrials):
        isingModel.Spins = InitialConfiguration
        isingModel.SetSeed()
        for n in range(MaxSteps + 1):
            isingModel.Temperature = initialTemperature * 0.99 ** n
            #isingModel.Temperature = 100 * np.exp(-0.005 * n)
            isingModel.Update()
            samples[n, 0] = n
            samples[n, 1] = isingModel.Temperature
            samples[n, 2] = isingModel.Energy
            samples[n, 3] = isingModel.EnergyOnBipartiteGraph
        minimumEnergiesData = np.append(minimumEnergiesData, samples[:, 2].min())

    print('Mean: {}'.format(np.mean(minimumEnergiesData)))
    print('Standard deviation: {}'.format(np.std(minimumEnergiesData)))
    print('Mode: {}'.format(stats.mode(minimumEnergiesData)))
    print('Minimum: {}'.format(np.min(minimumEnergiesData)))

    fig = plt.figure(figsize=(7, 3), dpi=200)
    ax = fig.add_subplot(121, xlabel='MC steps', ylabel='Energy')
    ax.grid()
    ax.plot(samples[:, 0], samples[:, 2], label='Original Hamiltonian')
    ax.plot(samples[:, 0], samples[:, 3], label='Double Hamiltonian')
    ax.legend()
    ax = fig.add_subplot(122, xlabel='Energy', ylabel='Frequency')
    ax.grid(which='both')
    ax.hist(minimumEnergiesData, bins=30)
    fig.suptitle(isingModel.Algorithm.name)
    plt.subplots_adjust(wspace=0.3)
    plt.show()

In [ ]:
def GenerateSquareLatticeEdges(numNodes):
    columns = math.ceil(math.sqrt(numNodes))
    result = {}
    for i in range(numNodes - 1):
        if (i + 1) % columns > 0:
            result[(i, i + 1)] = -1
        if (i + columns) < numNodes:
            result[(i, i + columns)] = -1
    return result

quadratic = GenerateSquareLatticeEdges(NumNodes)
isingModel = simulator.IsingModel({}, quadratic)
T0 = 2.e0 * np.sum([np.abs(J) for J in quadratic.values()])

In [ ]:
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
isingModel.Algorithm = simulator.Algorithms.fcSCA
isingModel.PinningParameter = 0.125e0 * isingModel.CalcLargestEigenvalue()
isingModel.FlipTrialRate = 0.8e0
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
def GenerateCompleteGraphEdges(numNodes):
    return {(i, j): -1 for i in range(numNodes) for j in range(i + 1, numNodes)}

quadratic = GenerateCompleteGraphEdges(NumNodes)
isingModel = simulator.IsingModel({}, quadratic)
T0 = 2.e0 * np.sum([np.abs(J) for J in quadratic.values()])

In [ ]:
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
isingModel.Algorithm = simulator.Algorithms.fcSCA
#isingModel.PinningParameter = 0.125e0 * isingModel.CalcLargestEigenvalue()
isingModel.FlipTrialRate = 0.3e0
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
OccupationProbability = 0.5e0
SeedForRandomGraph = 2048

def GenerateErdosRenyiEdges(numNodes, probability):
    rng = np.random.Generator(np.random.MT19937(SeedForRandomGraph))
    return {(i, j): -1 if rng.random() <= probability else 0 for i in range(numNodes) for j in range(i + 1, numNodes)}

quadratic = GenerateErdosRenyiEdges(NumNodes, OccupationProbability)
isingModel = simulator.IsingModel({}, quadratic)
T0 = 2.e0 * np.sum([np.abs(J) for J in quadratic.values()])

In [ ]:
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
isingModel.Algorithm = simulator.Algorithms.fcSCA
isingModel.PinningParameter = 0.e0
isingModel.FlipTrialRate = 0.2e0
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

## Dynamics for the propotion to spins

Consider the antiferromagnet $J_{x, y} = -1$ with $h_x = 0$ and $q_x = 0$ on a complete graph.
Taking the low temerature limit,
$$
    \lim_{\beta\uparrow\infty} P_\epsilon(\sigma, \tau)
    = \prod_{x\in D_{\sigma, \tau}} \left(\epsilon \left(\frac{1}{2} \mathbf{1}_{\{\sum_{y\sim x} \sigma_x \sigma_y = 0\}} + \mathbf{1}_{\{\sum_{y\sim x} \sigma_x \sigma_y > 0\}}\right)\right)
        \prod_{x\in D_{\sigma, \tau}^\complement} \left(1 - \epsilon \left(\frac{1}{2} \mathbf{1}_{\{\sum_{y\sim x} \sigma_x \sigma_y = 0\}} + \mathbf{1}_{\{\sum_{y\sim x} \sigma_x \sigma_y > 0\}}\right)\right).
$$
Let $a_t$ be the proportion of up spins to all spins.
The dynamics of $\{a_t\}_{t=0}^{\infty}$ is given as
$$
    a_{t+1} = \begin{cases}
        \left(1 - \frac{\epsilon}{2}\right) a_t & \left[\frac{1}{2} < a_t \leq \frac{1}{2} \left(1 + \frac{1}{\lvert V\rvert}\right)\right],\\
        \left(1 - \epsilon\right) a_t & \left[\frac{1}{2} \left(1 + \frac{1}{\lvert V\rvert}\right) < a_t < 1\right],\\
        \frac{\epsilon}{2} + \left(1 - \frac{\epsilon}{2}\right) a_t & \left[\frac{1}{2} \left(1 - \frac{1}{\lvert V\rvert}\right) \leq a_t < \frac{1}{2}\right],\\
        \epsilon + \left(1 - \epsilon\right) a_t & \left[0 < a_t < \frac{1}{2} \left(1 - \frac{1}{\lvert V\rvert}\right)\right].
    \end{cases}
$$

In [ ]:
flipTrialRate = 0.6e0
cardVertices = 10

def spinPropotion(previous):
    if 0 <= previous < 0.5e0 * (1 - 1 / cardVertices):
        return flipTrialRate + (1 - flipTrialRate) * previous
    elif 0.5e0 * (1 - 1 / cardVertices) <= previous < 0.5e0:
        return 0.5e0 * flipTrialRate + (1 - 0.5e0 * flipTrialRate) * previous
    elif 0.5e0 < previous <= 0.5e0 * (1 + 1 / cardVertices):
        return (1 - 0.5e0 * flipTrialRate) * previous
    elif 0.5e0 * (1 + 1 / cardVertices) < previous <= 1:
        return (1 - flipTrialRate) * previous
    else:
        raise ValueError('The value must be in the interval [0, 1]')

uSpinPropotion = np.frompyfunc(spinPropotion, 1, 1)

In [ ]:
x = np.linspace(0, 1, 100)
y = uSpinPropotion(x)

for i in range(100):
    y = uSpinPropotion(y)

plt.plot(x, y)
plt.plot(x, uSpinPropotion(y))
plt.xlabel('Up spin propotion')
plt.ylabel('Next propotion')
plt.grid()
plt.show()